# Entrada y salida, decoradores, y errores 

## Funciones que aceptan y devuelven funciones (Decoradores)

Vimos que las funciones pueden tomar como argumento otra función, pueden devolver una función, y también pueden hacer las dos cosas simultáneamente.

Consideremos la siguiente función `mas_uno`, que toma como argumento una función y devuelve otra función.

In [1]:
def mas_uno(func):
  "Devuelve una función"
  def fun(args):
    "Agrega 1 a cada uno de los elementos y luego aplica la función"
    xx = [x+1 for x in args]
    y= func(xx)
    return y
  return fun

In [2]:
ssum= mas_uno(sum)              # ssum es una función
mmin= mas_uno(min)              # mmin es una función
mmax= mas_uno(max)              # mmax es una función

In [3]:
a = [0, 1, 3.3, 5, 7.5, 2.2]
print(a)
print(sum(a), ssum(a))
print(min(a), mmin(a))
print(max(a), mmax(a))



[0, 1, 3.3, 5, 7.5, 2.2]
19.0 25.0
0 1
7.5 8.5


Podemos aplicar la función tanto a funciones "intrínsecas" como a funciones definidas por nosotros

In [4]:
def parabola(v):
  return [x**2 + 2*x for x in v]


In [5]:
mparabola = mas_uno(parabola)

In [6]:
print(parabola(a))
print(mparabola(a))

[0, 3, 17.49, 35, 71.25, 9.240000000000002]
[3, 8, 27.089999999999996, 48, 89.25, 16.64]


Notemos que al decorar una función estamos creando una enteramente nueva

In [7]:
del parabola                    # Borramos el objeto

In [8]:
parabola(a)

NameError: name 'parabola' is not defined

In [9]:
mparabola(a)

[3, 8, 27.089999999999996, 48, 89.25, 16.64]

Algunas veces queremos simplemente modificar la función original


### Notación para decoradores

Al procedimiento de modificar una función original, para crear una nueva y renombrar la nueva con el nombre de la original se lo conoce como "decorar", y tiene una notación especial

In [10]:
def parabola(v):
  return [x**2 + 2*x for x in v]
mparabola = mas_uno(parabola)
del parabola
parabola = mparabola
del mparabola

Son un montón de líneas, no todas necesarias, y podemos simplificarlo:

In [11]:
def parabola(v):
  return [x**2 + 2*x for x in v]
parabola = mas_uno(parabola)

Como esta es una situación que puede darse frecuentemente en algunas áreas se decidió simplificar la notación, introduciendo el uso de `@`. Lo anterior puede escribirse como:

In [12]:
@mas_uno
def mi_parabola(v):
  return [x**2 + 2*x for x in v]

La única restricción para utilizar esta notación es que la línea con el decorador debe estar inmediatamente antes de la definición de la función a decorar

In [13]:
mi_parabola(a)

[3, 8, 27.089999999999996, 48, 89.25, 16.64]


### Algunos usos de decoradores

Un uso común de los decoradores es agregar código de verificación de los argumentos de las funciones. Por ejemplo, la función que definimos anteriormente falla si le damos un punto `x` y queremos obtener el valor `y` de la parábola (más uno):

In [14]:
mi_parabola(3)

TypeError: 'int' object is not iterable

El problema aquí es que definimos la función para tomar como argumentos una lista (o al menos un iterable de números) y estamos tratando de aplicarla a un único valor.
Podemos definir un decorador para asegurarnos que el tipo es correcto (aunque esta implementación podría ser mejor)

In [15]:
def test_argumento_list_num(f):
  def check(v):
    if (type(v) == list):
      return f(v)
    else:
      print("Error: El argumento debe ser una lista")
  return check

In [16]:
mi_parabola = test_argumento_list_num(mi_parabola)

In [17]:
mi_parabola(3)

Error: El argumento debe ser una lista


In [18]:
mi_parabola(a)

[3, 8, 27.089999999999996, 48, 89.25, 16.64]

Supongamos que queremos simplemente extender la función para que sea válida también con argumentos escalares. Definimos una nueva función que utilizaremos como decorador

In [19]:
def hace_argumento_list(f):
  def check(v):
    "Corrige el argumento si no es una lista"
    if (type(v) == list):
      return f(v)
    else:
      return f([v])
  return check  

In [20]:
@hace_argumento_list
def parabola(v):
  return [x**2 + 2*x for x in v]


Esto es equivalente a:

In [21]:
def parabola(v):
  return [x**2 + 2*x for x in v]
parabola = hace_argumento_list(parabola)

In [22]:
print(parabola(3))
print(parabola([3]))

[15]
[15]
